In [1]:
import numpy as np
import tensorflow as tf

In [2]:
train_data = np.load("train_data.npy")
train_label = np.load("train_label.npy") 
# train_id = np.load("train_id.npy")

In [40]:
y_train = list(map(lambda x: 0 if x == 'low' else (1 if x == 'medium' else 2), train_label))

In [43]:
y_train_list = np.zeros((len(y_train), 3))
for i in range(len(y_train)):
    y_train_list[i, y_train[i]] = 1

In [49]:
y_train_list = y_train_list.astype(np.float32)

In [3]:
train_data = train_data.astype(np.float32)
train_data.dtype

dtype('float32')

### Default tensorflow variable dtype is tf.float32
### Placeholder dtype must be consitent with the feed_dict dtype, even tf.float32 != np.float32

In [ ]:
#sess = tf.InteractiveSession()

In [26]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME', use_cudnn_on_gpu=False)

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def max_pool_3x4(x):
    return tf.nn.max_pool(x, ksize=[1, 3, 4, 1], strides=[1, 3, 4, 1], padding='SAME')

def max_pool_4x4(x):
    return tf.nn.max_pool(x, ksize=[1, 4, 4, 1], strides=[1, 4, 4, 1], padding='SAME')

## Define input variable

In [5]:
x = tf.placeholder(shape=[None, 480, 640, 3], dtype="float")
y_ = tf.placeholder(shape=[None, 3], dtype="float")

## Define ConNet weight and bias

In [22]:
W_conv1 = weight_variable([10, 10, 3, 16]) # 5 by 5 is the kernel size. 1 is the input channel and 32 is the output channel
b_conv1 = bias_variable([16])

keep_prob = tf.placeholder(dtype="float")
x_drop = tf.nn.dropout(x, keep_prob)
h_conv1 = tf.nn.relu(conv2d(x_drop, W_conv1) + b_conv1)
h_pool1 = max_pool_4x4(h_conv1)

In [23]:
W_conv2 = weight_variable([10, 10, 16, 16])
b_conv2 = bias_variable([16])

keep_prob_2 = tf.placeholder(dtype="float")
h_pool1_drop = tf.nn.dropout(h_pool1, keep_prob_2)
h_conv2 = tf.nn.relu(conv2d(h_pool1_drop, W_conv2) + b_conv2)
h_pool2 = max_pool_4x4(h_conv2)

In [27]:
W_conv3 = weight_variable([10, 10, 16, 32])
b_conv3 = bias_variable([32])

keep_prob_3 = tf.placeholder(dtype="float")
h_pool2_drop = tf.nn.dropout(h_pool2, keep_prob_3)
h_conv3 = tf.nn.relu(conv2d(h_pool2_drop, W_conv3) + b_conv3)
h_pool3 = max_pool_3x4(h_conv3)

In [32]:
W_fc_4 = weight_variable([10*10*32, 1024])
b_fc_4 = bias_variable([1024])

h_pool3_flat = tf.reshape(h_pool3, [-1, 10*10*32])
y_fc_4 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc_4) + b_fc_4)

In [33]:
W_fc_5 = weight_variable([1024, 3])
b_fc_3 = bias_variable([3])

y_predict = tf.matmul(y_fc_4, W_fc_5) + b_fc_3

## Forward path

In [53]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_predict))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_predict,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for i in range(16):
    batch_temp = train_data[10*i: 10*(i+1)]
    label_temp = y_train_list[10*i: 10*(i+1)]
    train_accuracy = accuracy.eval(feed_dict={x:batch_temp, y_: label_temp, keep_prob: 1., keep_prob_2: 1., keep_prob_3: 1.})
    print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch_temp, y_: label_temp, keep_prob: 0.5, keep_prob_2: 0.5, keep_prob_3: 0.5})
    #with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #print(sess.run(y_predict, feed_dict={x: batch_temp, keep_prob: 0.5, keep_prob_2: 0.5, keep_prob_3: 0.5}))

step 0, training accuracy 0.5
step 1, training accuracy 0.5
step 2, training accuracy 1
step 3, training accuracy 0.9


In [28]:
h_conv3

<tf.Tensor 'Relu_6:0' shape=(?, 30, 40, 32) dtype=float32>

In [52]:
len(train_label)

167